In [ ]:
%load_ext autoreload
%autoreload 2
import penzai
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [ ]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it-q4_k.gguf", device_map="tpu:0",
                                         from_type="gemma",
                                        #  load_eager=True
                                         )

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("NousResearch/gemma-2b-it-tokenizer")
tokenizer.padding_side = "right"
format_prompt = """<start_of_turn>user\n
{}\n
<start_of_turn>model\n
{}"""

In [ ]:
from micrlhf.sampling import sample
sample(llama, tokenizer, format_prompt.format("Hi!", ""),
       batch_size=4, do_sample=True)[0]